Graph Plot


In [ ]:
import matplotlib.pyplot as plt


def roc_plot(y_test, y_pred):
    from sklearn.metrics import roc_curve, auc
    from sklearn.calibration import label_binarize
    from itertools import cycle

    rice = {
        0: 'Cammeo',
        1: 'Osmanscik',
    }

    y_test = label_binarize(y_test, classes=[0, 1])
    n_classes = y_test.shape[1]

    fpr, tpr, roc_auc = dict(), dict(), dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    colors = cycle(['red', 'blue'])

    plt.figure(figsize=(10, 5))

    for i, color in zip(range(n_classes), colors):
        plt.subplot(1, 2, i + 1)
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
                 label=f'{i}. {rice[i]} (area = {roc_auc[i]:.2f})')

        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([-.05, 1.05])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'Poker Hands ROC - {rice[i]}')
        plt.legend(loc='lower right')

    plt.tight_layout()
    plt.show(block=True)

Neural Network


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

cols = ['Area', 'Perimeter', 'Major_Axis_Length', 'Minor_Axis_Length',
        'Eccentricity', 'Convex_Area', 'Extent', 'Class']

data = pd.read_csv('./data/Rice_Cammeo_Osmancik.arff', names=cols)
data['Class'] = (data['Class'] == 'Osmancik').astype(int)

In [ ]:
from sklearn.preprocessing import StandardScaler

x = data.iloc[:, 0:7].values
y = data.iloc[:,-1]

scaler = StandardScaler()
x = scaler.fit_transform(x)

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=.3, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=.15, random_state=42)

y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=2)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=2)

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(64, input_shape=(7,), activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='tanh'))
model.add(tf.keras.layers.Dense(16, activation='tanh'))
model.add(tf.keras.layers.Dense(2, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])


model.fit(x_train, y_train, epochs=100, callbacks=tf.keras.callbacks.EarlyStopping(monitor='val_loss', verbose=1), validation_data=(x_val, y_val), verbose=1)
y_pred = model.predict(x_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

roc_plot(y_test, y_pred)

y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))